# Preprocessing

In [ ]:
!pip install jupyter-autotime
%load_ext autotime

In [ ]:
# ====================================================
# Directory settings
# ====================================================
import os

OUTPUT_DIR = 'input/inchi-preprocess'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    
if not os.path.exists('input/pytorch-image-models'):
    os.makedirs('input/pytorch-image-models')
    
if not os.path.exists('input/pytorch-image-models/pytorch-image-models-master'):
    os.makedirs('input/pytorch-image-models/pytorch-image-models-master')
    
if not os.path.exists('input/inchi-resnet-lstm-with-attention-starter'):
    os.makedirs('input/inchi-resnet-lstm-with-attention-starter')
    
if not os.path.isfile('train_labels.csv'):
    !wget https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/22422/2153105/compressed/train_labels.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1619975509&Signature=YuzUw%2FIcDEdnraZ0vyNS7q5q%2BT3c7DcBv8hV%2F%2FC0vqtYscLpal9ZBczcIPNamavvLbYQaMInivBGChSHO9ld1%2Bc6Wix5DnHAhWAo6EIimiIbk3Nbu%2F3%2BuU5H7pmZPwYiU0WHjviPmRrTbw24OctAYUbgrL9eSQGOgcgigQ88rI8wfPonZ4C%2Fk7fPizr6wtxDfRaG5TFe%2FkSNAf3NZvlydiAHlVknzozxaws%2FsW9I%2BTkPrJzNrjBUIz61Ync3EFaMAS2owdfGx7PDzHcKmYAHkAgBKTriO8Umd3bQrXJ56i%2FSpqZxNWzdEb1FceFKKOg8L%2BeP6lboxWkcbc38PBIx9A%3D%3D&response-content-disposition=attachment%3B+filename%3Dtrain_labels.csv.zip
    raise 'Please unzip the contents of this archive and place the train_labels.csv file in the root directory!'
    
if not os.path.isfile('sample_submission.csv'):
    !wget https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/22422/2153105/compressed/sample_submission.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1619975527&Signature=iaBM26uSwKKWMoLIG8Y3FZUFGccrp6C61Z5jz4HEqHruBPIzF7FeMtM3kNb7zN%2BIkp8bfOc4WMlvP%2FFApY1AkLUygtC9WiCW8QeQh5L5ob0HpIDAMa3cp3vXH6KRNTT80GnR6S67a%2FD%2B%2F11lUIWQibGb6jwkyE2hA5wuO3VpWvEd%2BV429kV3XizASQeD%2B2%2BGc2%2FjyQTu6ayAUnrVWluVTBAxdCKeKtsJ2%2B5d3kTkMZ6zqqSe30FKIyKrw1An4N7oBOYnsxy4CrqF4%2FcUasLCdGE6XLkfSmLj013Lh7LUkNOaKuk27FNce0%2Fq2Iw1lYc80P2pQAIKU0no9bVBHCWBdw%3D%3D&response-content-disposition=attachment%3B+filename%3Dsample_submission.csv.zip
    raise 'Please unzip the contents of this archive and place the sample_submissions.csv file in the root directory!'

In [ ]:
# ====================================================
# Library
# ====================================================
import os
import re
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import dill
tqdm.pandas()
import torch

# ====================================================
# Data Loading
# ====================================================
train = pd.read_csv('train_labels.csv')
print(f'train.shape: {train.shape}')

# ====================================================
# Preprocess functions
# ====================================================
def split_form(form):
    string = ''
    for i in re.findall(r"[A-Z][^A-Z]*", form):
        elem = re.match(r"\D+", i).group()
        num = i.replace(elem, "")
        if num == "":
            string += f"{elem} "
        else:
            string += f"{elem} {str(num)} "
    return string.rstrip(' ')

def split_form2(form):
    string = ''
    for i in re.findall(r"[a-z][^a-z]*", form):
        elem = i[0]
        num = i.replace(elem, "").replace('/', "")
        num_string = ''
        for j in re.findall(r"[0-9]+[^0-9]*", num):
            num_list = list(re.findall(r'\d+', j))
            assert len(num_list) == 1, f"len(num_list) != 1"
            _num = num_list[0]
            if j == _num:
                num_string += f"{_num} "
            else:
                extra = j.replace(_num, "")
                num_string += f"{_num} {' '.join(list(extra))} "
        string += f"/{elem} {num_string}"
    return string.rstrip(' ')

# ====================================================
# Tokenizer
# ====================================================
class Tokenizer(object):
    
    def __init__(self):
        self.stoi = {}
        self.itos = {}

    def __len__(self):
        return len(self.stoi)
    
    def fit_on_texts(self, texts):
        vocab = set()
        for text in texts:
            vocab.update(text.split(' '))
        vocab = sorted(vocab)
        vocab.append('<sos>')
        vocab.append('<eos>')
        vocab.append('<pad>')
        for i, s in enumerate(vocab):
            self.stoi[s] = i
        self.itos = {item[1]: item[0] for item in self.stoi.items()}
        
    def text_to_sequence(self, text):
        sequence = []
        sequence.append(self.stoi['<sos>'])
        for s in text.split(' '):
            sequence.append(self.stoi[s])
        sequence.append(self.stoi['<eos>'])
        return sequence
    
    def texts_to_sequences(self, texts):
        sequences = []
        for text in texts:
            sequence = self.text_to_sequence(text)
            sequences.append(sequence)
        return sequences

    def sequence_to_text(self, sequence):
        return ''.join(list(map(lambda i: self.itos[i], sequence)))
    
    def sequences_to_texts(self, sequences):
        texts = []
        for sequence in sequences:
            text = self.sequence_to_text(sequence)
            texts.append(text)
        return texts
    
    def predict_caption(self, sequence):
        caption = ''
        for i in sequence:
            if i == self.stoi['<eos>'] or i == self.stoi['<pad>']:
                break
            caption += self.itos[i]
        return caption
    
    def predict_captions(self, sequences):
        captions = []
        for sequence in sequences:
            caption = self.predict_caption(sequence)
            captions.append(caption)
        return captions

# Main

In [ ]:
# ====================================================
# main
# ====================================================
def main():
    # ====================================================
    # preprocess train.csv
    # ====================================================
    train['InChI_1'] = train['InChI'].progress_apply(lambda x: x.split('/')[1])
    train['InChI_text'] = train['InChI_1'].progress_apply(split_form) + ' ' + \
                            train['InChI'].apply(lambda x: '/'.join(x.split('/')[2:])).progress_apply(split_form2).values
    # ====================================================
    # create tokenizer
    # ====================================================
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(train['InChI_text'].values)
    torch.save(tokenizer, f"{OUTPUT_DIR}/tokenizer.pth", pickle_module=dill)
    print('Saved tokenizer')
    # ====================================================
    # preprocess train.csv
    # ====================================================
    lengths = []
    tk0 = tqdm(train['InChI_text'].values, total=len(train))
    for text in tk0:
        seq = tokenizer.text_to_sequence(text)
        length = len(seq) - 2
        lengths.append(length)
    train['InChI_length'] = lengths
    train.to_json(f"{OUTPUT_DIR}/train.json")
    print('Saved preprocessed train.json')

In [ ]:
if __name__ == '__main__':
    main()